In [1]:

import pandas as pd

modified_data = pd.read_excel('modified_data.xlsx')

modified_data.head()


import pandas as pd

# 读取并处理附件2的数据
csv_file_path = '附件/附件2.csv'
csv_data = pd.read_csv(csv_file_path, encoding='gbk')


csv_data.head()


,分拣中心,日期,小时,货量
0,SC54,2023/11/6,19,784
1,SC54,2023/11/21,5,72
2,SC54,2023/11/30,12,209
3,SC54,2023/11/13,1,520
4,SC54,2023/11/4,12,278


In [2]:
# 读取并处理Excel文件的未来货量预测数据
excel_file_path = 'Q1_未来30天各分拣中心预测货量数据.xlsx'
excel_data = pd.read_excel(excel_file_path)


excel_data.head()


,货量,日期,分拣中心
0,817.781638,1,SC48
1,816.782826,2,SC48
2,815.784013,3,SC48
3,814.785201,4,SC48
4,813.786388,5,SC48


做法1：请基于附件2数据计算各分拣中心每天每小时的平均货量分布，得出每天每小时的分布比例，接着基于未来30天各分拣中心的预测货量数据，按照比例分配各分拣中心未来每天每小时的货量

In [3]:
csv_data['日期'] = pd.to_datetime(csv_data['日期'])

# 计算每天每小时的平均货量
grouped_data = csv_data.groupby(['分拣中心', csv_data['日期'].dt.date, '小时']).agg({'货量': 'mean'}).reset_index()
grouped_data.columns = ['分拣中心', '日期', '小时', '平均货量']

# 计算每天的总货量
daily_totals = grouped_data.groupby(['分拣中心', '日期']).agg({'平均货量': 'sum'}).reset_index()
daily_totals.rename(columns={'平均货量': '日总货量'}, inplace=True)

# 计算每小时的分布比例
grouped_data = grouped_data.merge(daily_totals, on=['分拣中心', '日期'])
grouped_data['分布比例'] = grouped_data['平均货量'] / grouped_data['日总货量']

In [4]:
grouped_data

,分拣中心,日期,小时,平均货量,日总货量,分布比例
0,SC1,2023-11-01,0,6059.0,102024.0,0.059388
1,SC1,2023-11-01,1,4810.0,102024.0,0.047146
2,SC1,2023-11-01,2,5457.0,102024.0,0.053487
3,SC1,2023-11-01,3,5045.0,102024.0,0.049449
4,SC1,2023-11-01,4,4015.0,102024.0,0.039353
...,...,...,...,...,...,...
33276,SC9,2023-11-30,19,735.0,27321.0,0.026902
33277,SC9,2023-11-30,20,960.0,27321.0,0.035138
33278,SC9,2023-11-30,21,514.0,27321.0,0.018813
33279,SC9,2023-11-30,22,688.0,27321.0,0.025182


In [5]:
# 计算未来每天的总预测货量
forecast_totals = excel_data.groupby(['分拣中心', '日期']).agg({'货量': 'sum'}).reset_index()
forecast_totals.rename(columns={'货量': '预测日总货量'}, inplace=True)

# 准备未来每小时的预测数据
hours_ratios = grouped_data.groupby('小时').agg({'分布比例': 'mean'}).reset_index()
expanded_forecasts = forecast_totals.loc[forecast_totals.index.repeat(24)].reset_index(drop=True)
expanded_forecasts = pd.concat([expanded_forecasts, pd.DataFrame({
    '小时': list(range(24)) * len(forecast_totals),
    '平均分布比例': hours_ratios['分布比例'].tolist() * len(forecast_totals)
})], axis=1)

# 计算未来每小时的预测货量
expanded_forecasts['预测每小时货量'] = round(expanded_forecasts['预测日总货量'] * expanded_forecasts['平均分布比例'])


In [6]:
expanded_forecasts

,分拣中心,日期,预测日总货量,小时,平均分布比例,预测每小时货量
0,SC1,1,47881.814817,0,0.039580,1895.0
1,SC1,1,47881.814817,1,0.045133,2161.0
2,SC1,1,47881.814817,2,0.051535,2468.0
3,SC1,1,47881.814817,3,0.057866,2771.0
4,SC1,1,47881.814817,4,0.057207,2739.0
...,...,...,...,...,...,...
41035,SC9,30,27778.131084,19,0.060392,1678.0
41036,SC9,30,27778.131084,20,0.061282,1702.0
41037,SC9,30,27778.131084,21,0.066927,1859.0
41038,SC9,30,27778.131084,22,0.065190,1811.0


In [7]:
expanded_forecasts.to_excel('Q1_未来30天各分拣中心每小时预测货量数据_方法1.xlsx',index=None)

上面是对30天的每天每小时比例进行货量分配，现在采用直接按grouped_data的分布比例直接分配，不需要取均值

In [8]:
# 计算未来每天的总预测货量
forecast_totals = excel_data.groupby(['分拣中心', '日期']).agg({'货量': 'sum'}).reset_index()
forecast_totals.rename(columns={'货量': '预测日总货量'}, inplace=True)
grouped_data['日期']=pd.to_datetime(grouped_data['日期']).dt.day
# 准备未来每小时的预测数据
expanded_forecasts = forecast_totals.merge(grouped_data[['分拣中心', '日期', '小时', '分布比例']], on=['分拣中心', '日期'], how='left')

# 计算未来每小时的预测货量
expanded_forecasts['预测每小时货量'] = round(expanded_forecasts['预测日总货量'] * expanded_forecasts['分布比例'])

# 显示处理结果的前几行
expanded_forecasts.head()

,分拣中心,日期,预测日总货量,小时,分布比例,预测每小时货量
0,SC1,1,47881.814817,0,0.059388,2844.0
1,SC1,1,47881.814817,1,0.047146,2257.0
2,SC1,1,47881.814817,2,0.053487,2561.0
3,SC1,1,47881.814817,3,0.049449,2368.0
4,SC1,1,47881.814817,4,0.039353,1884.0


In [9]:
expanded_forecasts.to_excel('Q1_未来30天各分拣中心每小时预测货量数据_方法2.xlsx',index=None)